In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig

# Create and configure
config = PPOConfig().environment("CartPole-v1")

# Build and train
algo = config.build()
algo.train()

ModuleNotFoundError: No module named 'pkg_resources'